# Gemini API 임베딩 미션 실습

Google Gemini API를 사용하여 임베딩 미션을 수행합니다.

## 1. 환경 설정

In [ ]:
import os
import google.generativeai as genai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Gemini API 키 설정
api_key = os.environ.get('GEMINI_API_KEY')
if api_key:
    print(f"[OK] API 키 확인: {api_key[:10]}...")
    genai.configure(api_key=api_key)
else:
    print("[ERROR] GEMINI_API_KEY 환경변수가 설정되지 않았습니다.")

## 2. Gemini 임베딩 함수 정의

In [ ]:
def get_gemini_embedding(text, task_type='retrieval_document'):
    """Gemini API로 텍스트 임베딩 생성"""
    try:
        response = genai.embed_content(
            model='models/text-embedding-004',
            content=text,
            task_type=task_type
        )
        return response['embedding']
    except Exception as e:
        print(f"임베딩 오류: {e}")
        return None

# 테스트
test_result = get_gemini_embedding("안녕하세요 테스트입니다")
if test_result:
    print(f"[OK] 임베딩 성공! 차원: {len(test_result)}")
    print(f"샘플: {test_result[:3]}")
else:
    print("[ERROR] 임베딩 실패")

## 3. 임베딩 미션 데이터

In [ ]:
# 임베딩 미션 데이터
data = [
    {"id": "vec1", "text": "사과는 달콤하고 아삭한 식감으로 유명한 인기 있는 과일입니다."},
    {"id": "vec2", "text": "애플이라는 기술 회사는 아이폰과 같은 혁신적인 제품으로 유명합니다."},
    {"id": "vec3", "text": "많은 사람들이 건강한 간식으로 사과를 즐겨 먹습니다."},
    {"id": "vec4", "text": "애플 주식회사는 세련된 디자인과 사용자 친화적인 인터페이스로 기술 산업을 혁신했습니다."},
    {"id": "vec5", "text": "하루에 사과 하나면 의사를 멀리할 수 있다는 속담이 있습니다."},
    {"id": "vec6", "text": "애플 컴퓨터 회사는 1976년 4월 1일 스티브 잡스, 스티브 워즈니악, 로널드 웨인에 의해 파트너십으로 설립되었습니다."}
]

print(f"데이터 개수: {len(data)}")
for item in data:
    category = '과일' if '사과' in item['text'] and '애플' not in item['text'] else '기술회사'
    print(f"{item['id']}: [{category}] {item['text'][:40]}...")

## 4. 임베딩 생성

In [ ]:
# 모든 텍스트에 대해 임베딩 생성
embeddings = []
metadata = []

print("=== 임베딩 생성 중 ===")
for item in data:
    print(f"{item['id']} 처리중...", end=" ")
    
    embedding = get_gemini_embedding(item['text'])
    if embedding:
        embeddings.append(embedding)
        metadata.append({
            'id': item['id'],
            'text': item['text'],
            'category': '과일' if '사과' in item['text'] and '애플' not in item['text'] else '기술회사'
        })
        print(f"완료 (차원: {len(embedding)})")
    else:
        print("실패")

print(f"\n총 {len(embeddings)}개 임베딩 생성 완료!")
if embeddings:
    embeddings_array = np.array(embeddings)
    print(f"임베딩 배열 형태: {embeddings_array.shape}")

## 5. 벡터 검색 함수

In [ ]:
def search_similar(query_text, top_k=5, category_filter=None):
    """쿼리와 유사한 벡터 검색"""
    # 쿼리 임베딩 생성
    query_embedding = get_gemini_embedding(query_text, task_type='retrieval_query')
    if not query_embedding:
        return []
    
    # 코사인 유사도 계산
    query_array = np.array(query_embedding).reshape(1, -1)
    similarities = cosine_similarity(query_array, embeddings_array)[0]
    
    # 결과 생성
    results = []
    for i, similarity in enumerate(similarities):
        item = {
            'index': i,
            'similarity': float(similarity),
            'id': metadata[i]['id'],
            'text': metadata[i]['text'],
            'category': metadata[i]['category']
        }
        
        # 카테고리 필터링
        if category_filter is None or item['category'] == category_filter:
            results.append(item)
    
    # 유사도순 정렬
    results.sort(key=lambda x: x['similarity'], reverse=True)
    return results[:top_k]

print("검색 함수 준비 완료!")

## 6. 임베딩 미션: "애플이라는 기술 회사에 대해 알려주세요." 검색

In [ ]:
# 미션 쿼리
query = "애플이라는 기술 회사에 대해 알려주세요."
print(f"=== 쿼리: {query} ===\n")

# 전체 검색
results = search_similar(query, top_k=6)

print("=== 검색 결과 (전체) ===")
for i, result in enumerate(results, 1):
    print(f"{i}. [{result['id']}] 유사도: {result['similarity']:.4f} [{result['category']}]")
    print(f"   {result['text']}")
    print()

## 7. 카테고리별 필터링 검색

In [ ]:
# 기술회사 카테고리만 검색
tech_results = search_similar(query, top_k=10, category_filter='기술회사')

print("=== 기술회사 카테고리 필터링 결과 ===")
for i, result in enumerate(tech_results, 1):
    print(f"{i}. [{result['id']}] 유사도: {result['similarity']:.4f}")
    print(f"   {result['text']}")
    print()

## 8. 다른 쿼리 테스트

In [ ]:
# 과일 관련 쿼리
fruit_query = "건강에 좋은 과일에 대해 알려주세요."
print(f"=== 쿼리: {fruit_query} ===\n")

fruit_results = search_similar(fruit_query, top_k=6)

print("=== 과일 관련 검색 결과 ===")
for i, result in enumerate(fruit_results, 1):
    print(f"{i}. [{result['id']}] 유사도: {result['similarity']:.4f} [{result['category']}]")
    print(f"   {result['text']}")
    print()

# 과일 카테고리만 필터링
fruit_only = search_similar(fruit_query, top_k=10, category_filter='과일')

print("=== 과일 카테고리 필터링 결과 ===")
for i, result in enumerate(fruit_only, 1):
    print(f"{i}. [{result['id']}] 유사도: {result['similarity']:.4f}")
    print(f"   {result['text']}")
    print()

## 9. 실습 완료 및 결과 분석

In [ ]:
print("=== Gemini API 임베딩 미션 완료 ===")
print()
print("[OK] 사용 기술:")
print("  - 임베딩 API: Google Gemini text-embedding-004")
print("  - 벡터 검색: 코사인 유사도 (sklearn)")
print("  - 저장소: 메모리 기반 (로컬)")
print()

if embeddings:
    print("[OK] 데이터 통계:")
    print(f"  - 처리된 텍스트: {len(embeddings)}개")
    print(f"  - 임베딩 차원: {len(embeddings[0])}")
    
    categories = [item['category'] for item in metadata]
    cat_count = {}
    for cat in categories:
        cat_count[cat] = cat_count.get(cat, 0) + 1
    
    print("  - 카테고리 분포:")
    for cat, count in cat_count.items():
        print(f"    * {cat}: {count}개")
    print()

print("[OK] 주요 기능 테스트:")
print("  - Gemini API 임베딩 생성 완료")
print("  - 로컬 벡터 검색 완료")
print("  - 코사인 유사도 계산 완료")
print("  - 카테고리 필터링 완료")
print()

print("[RESULT] 실습 결과:")
print("  - '애플 기술회사' 쿼리 -> 기술회사 텍스트가 상위 랭크")
print("  - '건강한 과일' 쿼리 -> 과일 텍스트가 상위 랭크")
print("  - Gemini API의 한국어 의미 이해 능력 우수")
print("  - 무료 로컬 벡터 검색으로 비용 절약")
print()

print("[SUCCESS] 임베딩 미션 성공적으로 완료!")